# Operator change of HQ

## Setup

In [1]:
library(tidyverse)
library(oildata)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.1
✔ tidyr   1.1.1     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
pipelines <- filter(pipelines, on_offshore == "onshore" & commodity %in% c("crude", "rpp"))
head(pipelines)

year,commodity,ID,name,state,offshore_share,on_offshore,hca,miles,volume_crude,⋯,estimate_volume_specific,incidents,significant_incidents,serious_incidents,incidents_volume,recovered,net_loss_volume,significant_incidents_volume,incidents_cost,significant_incidents_cost
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2008,crude,31083,Helco-Hawaii Electric Light Company,NA,0,onshore,0.00,2.18,1595583,⋯,1595583,0,0,0,0,0,0,0,0.00,0.00
2008,rpp,26111,United Refining,PA,0,onshore,1.50,1.50,0,⋯,1400091,0,0,0,0,0,0,0,0.00,0.00
2008,crude,30658,Kern Oil & Refining,NA,0,onshore,18.57,18.57,122000000,⋯,122000000,0,0,0,0,0,0,0,0.00,0.00
2006,rpp,Amoco (Group),Amoco Oil,NA,0,onshore,728.00,1743.00,0,⋯,16148390168,0,0,0,0,0,0,0,0.00,0.00
2006,crude,31874,"Tesoro Refining & Marketing Co., Golden Eagle",NA,0,onshore,4.30,4.30,140259924,⋯,140259924,0,0,0,0,0,0,0,0.00,0.00
2006,crude,32103,Crimson Pipeline,NA,0,onshore,307.27,307.27,1148936999,⋯,1148936999,1,1,0,2,1,1,2,58634.38,58634.38


## Change of HQ

We track change of HQ from one year to another. Particularly of interest are companies moving to Texas or Louisiana, which have particularly lax regulations.

In [3]:
change_state <- function(df) {
    brown_states <- c("TX", "LA")
    df %>%
        group_by(ID, commodity) %>%
        arrange(desc(year)) %>%
        mutate(move = state != lag(state)) %>%
        mutate(move_tx_la = case_when(is.na(move) ~ NA, 
                                      move & !(lag(state) %in% brown_states) & state %in% brown_states ~ TRUE, 
                                      TRUE ~ FALSE)) %>%
        mutate(moved_tx_la = move_tx_la | lag(move_tx_la) | lag(move_tx_la, 2))
}

pipelines <- change_state(pipelines)
pipelines_ungrouped <- change_state(pipelines_ungrouped)

In [4]:
head(select(pipelines_ungrouped, move, move_tx_la, everything()))

move,move_tx_la,year,commodity,ID,name,state,offshore_share,on_offshore,hca,⋯,incidents,significant_incidents,serious_incidents,incidents_volume,recovered,net_loss_volume,significant_incidents_volume,incidents_cost,significant_incidents_cost,moved_tx_la
<lgl>,<lgl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,⋯,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
NA,NA,2019,CO2,117,Air Products & Chemicals,PA,0,total,11.72,⋯,0,0,0,0.00,0.0,0.00,0,0.0,0.0,NA
FALSE,FALSE,2019,CO2,117,Air Products & Chemicals,PA,0,offshore,0.00,⋯,0,0,0,0.00,0.0,0.00,0,0.0,0.0,NA
FALSE,FALSE,2019,CO2,117,Air Products & Chemicals,PA,0,onshore,11.72,⋯,0,0,0,0.00,0.0,0.00,0,0.0,0.0,NA
NA,NA,2019,crude,300,Plains Pipeline,TX,0,total,2022.67,⋯,19,7,0,1038.12,956.5,81.62,907,536838.3,377120.1,NA
FALSE,FALSE,2019,crude,300,Plains Pipeline,TX,0,offshore,0.00,⋯,0,0,0,0.00,0.0,0.00,0,0.0,0.0,NA
FALSE,FALSE,2019,crude,300,Plains Pipeline,TX,0,onshore,2022.67,⋯,19,7,0,1038.12,956.5,81.62,907,536838.3,377120.1,NA


In [5]:
table(is.na(pipelines_ungrouped$move))


FALSE  TRUE 
20088  4218 

In [6]:
table(pipelines_ungrouped$move_tx_la)


FALSE  TRUE 
20015    73 

In [7]:
get_volume <- pipelines_ungrouped %>%
    group_by(ID, commodity) %>%
    filter(TRUE %in% move_tx_la) %>%
    arrange(year) %>%
    mutate(prev_spill = sum(lag(incidents_volume), lag(incidents_volume, 2), lag(incidents_volume, 3), na.rm = T), 
           future_spill = sum(incidents_volume, lead(incidents_volume), lead(incidents_volume, 2), na.rm = T)) %>%
    filter(move_tx_la)
sum(get_volume$prev_spill)
sum(get_volume$future_spill)

[1] 292287

[1] 288166.3